In [118]:
import pandas as pd
import re
import os

In [119]:
df = pd.read_csv(os.path.join('budget-data', 'budget-data.csv'))
df.head()

,company_name,quote_id,description,qty,unit_price
0,Municipio do Seixal,22290,Filtros MV/G4.445.622.44,2.0,11.50
1,Municipio do Seixal,22290,Filtros MV/G4.391.622.44,1.0,11.25
2,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45
3,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45
4,Municipio do Seixal,22290,Filtros MV/G4.448.448.48,2.0,10.25


In [120]:
df_old = df
df = df_old.copy()

In [121]:
df = df_old.copy()

## Extract filter types

In [122]:
def extract_filter_type(description):
    regex_filter_types = re.compile(r'\s?\.?([G|M|F|H]\d+)\.?\s?')
    groups = re.findall(regex_filter_types, description)
    
    return groups[0] if len(groups) > 0 else None

filter_types = df.description.apply(extract_filter_type)
df['filter_efficiency'] = filter_types

In [123]:
df.head()

,company_name,quote_id,description,qty,unit_price,filter_efficiency
0,Municipio do Seixal,22290,Filtros MV/G4.445.622.44,2.0,11.50,G4
1,Municipio do Seixal,22290,Filtros MV/G4.391.622.44,1.0,11.25,G4
2,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45,F7
3,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45,F7
4,Municipio do Seixal,22290,Filtros MV/G4.448.448.48,2.0,10.25,G4


## Extract item types

In [124]:
def extract_type(i):
    if 'filtro' in i.lower():
        return 'Filtro'
    elif 'rolo' in i.lower():
        return 'Rolo'
    elif 'manta' in i.lower():
        return 'Manta'
    else:
        return None

df['item_type'] = df['description'].apply(extract_type)

In [125]:
df

,company_name,quote_id,description,qty,unit_price,filter_efficiency,item_type
0,Municipio do Seixal,22290,Filtros MV/G4.445.622.44,2.0,11.50,G4,Filtro
1,Municipio do Seixal,22290,Filtros MV/G4.391.622.44,1.0,11.25,G4,Filtro
2,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45,F7,Filtro
3,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45,F7,Filtro
4,Municipio do Seixal,22290,Filtros MV/G4.448.448.48,2.0,10.25,G4,Filtro
...,...,...,...,...,...,...,...
782,ATM,18579,Filtros MV/G4.630.495.48,2.0,7.80,G4,Filtro
783,ATM,18579,Filtros MV/G4.875.670.145,1.0,27.50,G4,Filtro
784,ATM,18579,Filtros F7.592.592.500.25.7,13.0,14.95,F7,Filtro
785,ATM,18579,Filtros F7.287.592.500.25.4,2.0,11.95,F7,Filtro


## Extract filter sizes

In [126]:
def extract_raw_dimensions(description):
    regex_dimensions = re.compile(r'[\s|\.]((?:\d+\.)+(?:\d+))')
    
    matches = re.findall(regex_dimensions, description)
    
    return '.'.join(matches) if len(matches) > 0 else None

df['dimensions'] = df['description'].apply(extract_raw_dimensions)


In [127]:
def extract_refined_dimensions(raw_dimension):
    if raw_dimension is None:
        return None
    
    dimensions = raw_dimension.split('.')
    
    if len(dimensions) == 3:
        return {
            'L': int(dimensions[0]),
            'H': int(dimensions[1]),
            'G': int(dimensions[2]),
            'D': None,
            'P': None
        }
    elif len(dimensions) == 5:
        return {
            'L': int(dimensions[0]),
            'H': int(dimensions[1]),
            'G': int(dimensions[3]),
            'D': int(dimensions[2]),
            'P': int(dimensions[4])
        }
    else:
        return None
    
refined_dimensions: dict = df['dimensions'].apply(extract_refined_dimensions)

length_values = refined_dimensions.apply(lambda x: x['L'] if x is not None else None)
height_values = refined_dimensions.apply(lambda x: x['H'] if x is not None else None)
gutter_dimensions = refined_dimensions.apply(lambda x: x['G'] if x is not None else None)
depth_values = refined_dimensions.apply(lambda x: x['D'] if x is not None else None)
pockets_values = refined_dimensions.apply(lambda x: x['P'] if x is not None else None)

df['Length'] = length_values
df['Height'] = height_values
df['Gutter'] = gutter_dimensions
df['Depth'] = depth_values
df['Pockets'] = pockets_values

In [136]:
df

,company_name,quote_id,description,qty,unit_price,filter_efficiency,item_type,dimensions,Length,Height,Gutter,Depth,Pockets
0,Municipio do Seixal,22290,Filtros MV/G4.445.622.44,2.0,11.50,G4,Filtro,445.622.44,445.0,622.0,44.0,NaN,NaN
1,Municipio do Seixal,22290,Filtros MV/G4.391.622.44,1.0,11.25,G4,Filtro,391.622.44,391.0,622.0,44.0,NaN,NaN
2,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45,F7,Filtro,592.592.520.25.10,592.0,592.0,25.0,520.0,10.0
3,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45,F7,Filtro,592.592.520.25.10,592.0,592.0,25.0,520.0,10.0
4,Municipio do Seixal,22290,Filtros MV/G4.448.448.48,2.0,10.25,G4,Filtro,448.448.48,448.0,448.0,48.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,ATM,18579,Filtros MV/G4.630.495.48,2.0,7.80,G4,Filtro,630.495.48,630.0,495.0,48.0,NaN,NaN
783,ATM,18579,Filtros MV/G4.875.670.145,1.0,27.50,G4,Filtro,875.670.145,875.0,670.0,145.0,NaN,NaN
784,ATM,18579,Filtros F7.592.592.500.25.7,13.0,14.95,F7,Filtro,592.592.500.25.7,592.0,592.0,25.0,500.0,7.0
785,ATM,18579,Filtros F7.287.592.500.25.4,2.0,11.95,F7,Filtro,287.592.500.25.4,287.0,592.0,25.0,500.0,4.0


# Experimental

## Differentiate between 'normal' 3-dimension filter and 5-dimension filter

- 3-dimension filter will be called: standard_filter
- 5-dimension filter will be called: pocket_filter 

In [137]:
def extract_filter_type_from_dimensions(item):
    if item['dimensions'] is None:
        return item['item_type']
    
    dimension_count = len(item['dimensions'].split('.'))
    
    if dimension_count == 3:
        return 'standard_filter'
    elif dimension_count == 5:
        return 'pocketed_filter'
    else:
        return item['item_type']

df_experimental = df.copy()
df_experimental['item_type'] = df_experimental.apply(extract_filter_type_from_dimensions, axis=1)

In [138]:
df_experimental.head()

,company_name,quote_id,description,qty,unit_price,filter_efficiency,item_type,dimensions,Length,Height,Gutter,Depth,Pockets
0,Municipio do Seixal,22290,Filtros MV/G4.445.622.44,2.0,11.50,G4,standard_filter,445.622.44,445.0,622.0,44.0,NaN,NaN
1,Municipio do Seixal,22290,Filtros MV/G4.391.622.44,1.0,11.25,G4,standard_filter,391.622.44,391.0,622.0,44.0,NaN,NaN
2,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45,F7,pocketed_filter,592.592.520.25.10,592.0,592.0,25.0,520.0,10.0
3,Municipio do Seixal,22290,Filtros F7.592.592.520.25.10,2.0,25.45,F7,pocketed_filter,592.592.520.25.10,592.0,592.0,25.0,520.0,10.0
4,Municipio do Seixal,22290,Filtros MV/G4.448.448.48,2.0,10.25,G4,standard_filter,448.448.48,448.0,448.0,48.0,NaN,NaN


In [139]:
df_experimental.item_type.value_counts()

item_type
standard_filter    472
pocketed_filter    251
Filtro              12
Rolo                 6
Manta                1
Name: count, dtype: int64

In [140]:
df_experimental[df_experimental['item_type'] == 'Filtro']

,company_name,quote_id,description,qty,unit_price,filter_efficiency,item_type,dimensions,Length,Height,Gutter,Depth,Pockets
269,Faclima,18591,Filtros MP/G3.(1095.185.15)/2,1.0,12.50,G3,Filtro,185.15,NaN,NaN,NaN,NaN,NaN
270,Faclima,18591,Filtros MP/G3.(895.185.15)/2,9.0,10.50,G3,Filtro,185.15,NaN,NaN,NaN,NaN,NaN
271,Faclima,18591,Filtros MP/G3.(695.185.15)/2,10.0,9.50,G3,Filtro,185.15,NaN,NaN,NaN,NaN,NaN
301,Lennox,22260,Filtros F7.287.592.670.7,2.0,16.50,F7,Filtro,287.592.670.7,NaN,NaN,NaN,NaN,NaN
302,Lennox,22260,Filtros F7.592.287.670.3,4.0,15.50,F7,Filtro,592.287.670.3,NaN,NaN,NaN,NaN,NaN
303,Lennox,22260,Filtros F7.592.592.520.6,16.0,17.50,F7,Filtro,592.592.520.6,NaN,NaN,NaN,NaN,NaN
304,Lennox,22260,Filtros F7.287.592.670.5,10.0,18.50,F7,Filtro,287.592.670.5,NaN,NaN,NaN,NaN,NaN
305,Lennox,22260,Filtros F7.592.592.670.10,10.0,24.50,F7,Filtro,592.592.670.10,NaN,NaN,NaN,NaN,NaN
306,Lennox,22260,Filtros F7.592.592.670.6,4.0,19.50,F7,Filtro,592.592.670.6,NaN,NaN,NaN,NaN,NaN
503,Race,18611,Filtros MV/G4.592.59248,200.0,7.45,G4,Filtro,592.59248,NaN,NaN,NaN,NaN,NaN
